In [1]:
import pandas as pd
import numpy as np

In [14]:
class DataGenerator:
    def __init__(self,
                 num_blocks=50):

        self.num_blocks = num_blocks
        self.df = pd.read_excel('../data/블록-계획데이터(예제)_수정.xlsx')

        # 없는 데이터를 전부 제거한 데이터프레임 생성(이후 이 데이터프레임을 사용)
        self.df_revised = self.df.dropna(axis=0)
        idx_1 = self.df_revised[self.df_revised['취부팀_코드'] == 'XXX'].index
        self.df_revised = self.df_revised.drop(idx_1)
        idx_2 = self.df_revised[self.df_revised['용접팀_코드'] == 'XXX'].index
        self.df_revised = self.df_revised.drop(idx_2)
        idx_3 = self.df_revised[self.df_revised['stage_코드'] == 'XXX'].index
        self.df_revised = self.df_revised.drop(idx_3)
        idx_4 = self.df_revised[self.df_revised['정반_코드'] == 'XXXX'].index
        self.df_revised = self.df_revised.drop(idx_4)
        idx_5 = self.df_revised[self.df_revised['실적공기'] < 0].index
        self.df_revised = self.df_revised.drop(idx_5)
        self.df_revised.reset_index(inplace=True)


    def generate_process(self):         # 공종 명칭 생성 함수, 공정이 나오는 비율에 맞춰서 데이터 생성
        df_process_count = pd.DataFrame(self.df_revised['공종_명칭'].value_counts())
        df_process_count.reset_index(inplace=True)
        df_process_count['Proportion'] = df_process_count['count'] / df_process_count['count'].sum()
        process_type = np.random.choice(df_process_count['공종_명칭'], p=df_process_count['Proportion'])
        return process_type

    def generate_group(self):       # 그룹을 선택한 후 선종과 블록 종류로 나누기 위한 함수
        # 그룹화를 위한 새로운 데이터프레임 생성
        df_revised_for_group = self.df_revised
        group_list = []
        for i in range(self.df_revised.shape[0]):
            group_code = self.df_revised.loc[i, '선종_코드'] + '_' + self.df_revised.loc[i, '블록'][0]
            group_list.append(group_code)
        df_revised_for_group['선종_블록'] = group_list
        df_group_count = pd.DataFrame(df_revised_for_group['선종_블록'].value_counts())
        df_group_count.reset_index(inplace=True)
        df_group_count['Proportion'] = df_group_count['count'] / df_group_count['count'].sum()

        # 그룹을 랜덤으로 선택->선종과 블록 타입으로 분리
        group = np.random.choice(df_group_count['선종_블록'], p=df_group_count['Proportion'])
        ship_type = group[0:2]
        block_type = group[-1]

        return (ship_type, block_type)


    def generate(self):
        columns = ["Block_Name", "Block_ID", "Process_Type", "Ship_Type", "Block_Type"]

        df_blocks = []

        for j in range(self.num_blocks):
            # 블록 이름과 id 생성
            name = "J-%d" % j
            id = j

            # 공종 명칭 생성
            process_type = self.generate_process()

            # 선종과 블록 타입 생성
            group_code = self.generate_group()          # column에 포함되지는 않음
            ship_type = group_code[0]
            block_type = group_code[1]

            row = [name, id, process_type, ship_type, block_type]
            df_blocks.append(row)

        df_blocks = pd.DataFrame(df_blocks, columns=columns)

        return df_blocks

In [15]:
generator = DataGenerator()
df_blocks = generator.generate()
df_blocks

,Block_Name,Block_ID,Process_Type,Ship_Type,Block_Type
0,J-0,0,Final조립,CN,B
1,J-1,1,곡중조,CN,D
2,J-2,2,Final조립,LP,D
3,J-3,3,곡중조,CN,E
4,J-4,4,곡중조,PT,S
5,J-5,5,Final조립,LN,B
6,J-6,6,Final조립,LP,E
7,J-7,7,곡중조,CN,E
8,J-8,8,Final조립,CN,F
9,J-9,9,곡중조,CN,S


In [4]:
df_blocks['Process_Type'].value_counts()

Process_Type
Final조립    17
곡중조        16
대조중조        9
평중조         8
Name: count, dtype: int64